In [ ]:
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [1]:
! pip install simpletransformers

     |████████████████████████████████| 221 kB 5.1 MB/s 
     |████████████████████████████████| 2.8 MB 33.5 MB/s 
     |████████████████████████████████| 3.3 MB 43.4 MB/s 
     |████████████████████████████████| 124 kB 44.0 MB/s 
     |████████████████████████████████| 1.7 MB 27.7 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |████████████████████████████████| 1.2 MB 28.8 MB/s 
     |████████████████████████████████| 8.0 MB 21.6 MB/s 
     |████████████████████████████████| 264 kB 52.4 MB/s 
     |████████████████████████████████| 636 kB 41.6 MB/s 
     |████████████████████████████████| 50 kB 6.2 MB/s 
     |████████████████████████████████| 895 kB 49.1 MB/s 
     |████████████████████████████████| 133 kB 51.3 MB/s 
     |████████████████████████████████| 170 kB 41.9 MB/s 
     |████████████████████████████████| 97 kB 7.1 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 243 kB 49.2 MB/s 
     |█████████████████

In [2]:
import pandas as pd 
import numpy as np
import os
from google.colab import files
import matplotlib.pyplot as plt
import sklearn

#others
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

from scipy.special import softmax
from sklearn.model_selection import train_test_split

#nn
from simpletransformers.classification import ClassificationModel
import collections

In [3]:
url = 'https://raw.githubusercontent.com/JHDatz/UnicodeProjects/main/Twitter%20Project/Data/clean.csv'
df = pd.read_csv(url)
df.dropna(inplace = True)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(df['clean'], df['injury_report'], random_state = 91, stratify = df['injury_report'], test_size = 0.2)

train = pd.concat([X_train, y_train], axis = 1)
test = pd.concat([X_test, y_test], axis = 1)
y = test['injury_report'].tolist()

train = train[['clean','injury_report']]
test = test[['clean','injury_report']]
tst = test['clean'].tolist()

print(collections.Counter(train["injury_report"]))
print(collections.Counter(test["injury_report"]), end = "\n\n")


train.reset_index().to_csv("train 06-09.csv")
files.download('train 06-09.csv')
test.reset_index().to_csv("test 06-09.csv")
files.download('test 06-09.csv')

In [13]:
train

,clean,injury_report
20581,It CRAZY quickly Giannis body recovered injury,1
12855,Darin Ruf original starting lineup delivered t...,0
22617,That fun let PacktheJoe httpstcokFFNRoNX,0
23133,orioles another undrafted free agent signing P...,0
23880,Hacksaws Mini Poll Lakers decision make guard ...,0
...,...,...
1047,Jonathan India scratched lineup right ankle so...,1
7832,Dont sleep TimAnderson httpstcookMtDaBd,0
15071,A woman choking Giants game father Nationals p...,0
11673,crissyfroyd Wait already point Nobrainer,0


In [ ]:
test

In [14]:
def evaluations(model, model_name):
  result, model_outputs, wrong_predictions = model.eval_model(test, acc=sklearn.metrics.accuracy_score)
  print("Result: ")
  print(result, end = "\n\n")
  
  df_model_outputs = pd.DataFrame(model_outputs, columns = ['P0','P1'])
  df_model_outputs.to_csv(model_name +'_model_outputs.csv')
  files.download(model_name +'_model_outputs.csv')
  print(model_name + 'outputs downloaded', end = '\n\n')

  print(wrong_predictions, end = '\n\n')
  
  fn = []
  fp = []
  for i in range(len(model_outputs)):
    x = model_outputs[i];
    if x[1]>x[0] and y[i]==0:
      fp.append([i,tst[i]])
    elif x[0]>x[1] and y[i]==1:
      fn.append([i,tst[i]])
  print(fp)
  print(len(fp))
  print(fn)
  print(len(fn))
  
  df_fp = pd.DataFrame(fp, columns = ['index','fp tweet'])
  df_fp.to_csv(model_name + '_fp_tweets.csv')
  files.download(model_name + '_fp_tweets.csv')

  df_fn = pd.DataFrame(fn, columns = ['index','fn tweet'])
  df_fn.to_csv(model_name + '_fn_tweets.csv')
  files.download(model_name + '_fn_tweets.csv')

  print(model_name + ' FP and FN tweet dataframes downloaded', end = '\n\n')

In [ ]:
def model_setup(arch, id):

  #initialise
  model =  ClassificationModel(arch, id, num_labels=2, use_cuda=True, args={
        'reprocess_input_data': True,
        'use_cached_eval_features': False,
        'overwrite_output_dir': True,
        'num_train_epochs': 5,
        'use_early_stopping': True,
        'early_stopping_delta': 0.01,
        'early_stopping_metric': "mcc",
        'early_stopping_metric_minimize': False,
        'early_stopping_patience': 5,
        'evaluate_during_training_steps': 1000
      })
  
  # print config
  print(model.config, end = '\n\n')

  #train
  print('Training starts')
  model.train_model(train)
  evaluations(model, id + '-5-epochs')
  
  return model

### DISTILBERT MODEL

In [ ]:
bert = model_setup('distilbert', 'distilbert-base-uncased')

### XLNET CONFIG

In [ ]:
bert2 = model_setup('xlnet', 'xlnet-base-cased')

### ROBERTA

In [ ]:
bert3 = model_setup('roberta', 'roberta-base')

### XLM-ROBERTA

In [ ]:
bert4 = model_setup('xlmroberta', 'xlm-roberta-base')

### Electra Model

In [ ]:
bert5 = model_setup('electra', 'google/electra-base-discriminator')

### distilbert-base-uncased-finetuned-sst-2-english

In [ ]:
bert6 = model_setup('distilbert', 'distilbert-base-uncased-finetuned-sst-2-english')